<a href="https://colab.research.google.com/github/danbernstein/parkingdirty/blob/master/parking_dirty_transfer_learning_with_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/danbernstein/parkingdirty.git 
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

# download tensorboard
#!pip install -q tf-nightly-2.0-preview --no-cache-dir
!pip install pascal-voc-writer


# Load the TensorBoard notebook extension
%reload_ext tensorboard

from matplotlib import pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import os
import matplotlib.patches as patches
import numpy as np

import tensorflow as tf

from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# build CNN using tf.keras
from tensorflow.keras import models # for building CNN (deep learning)
from tensorflow.keras import layers # for building fully connected network
from tensorflow.keras import losses
from tensorflow.keras import optimizers

import datetime

#exec(open("parkingdirty/object_detection/py/object_detection_functions.py").read())




Cloning into 'parkingdirty'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 542 (delta 21), reused 0 (delta 0), pack-reused 482
Receiving objects: 100% (542/542), 9.27 MiB | 23.05 MiB/s, done.
Resolving deltas: 100% (295/295), done.
Cloning into 'models'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 25872 (delta 1), reused 1 (delta 0), pack-reused 25865
Receiving objects: 100% (25872/25872), 508.53 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (15526/15526), done.
Checking out files: 100% (2907/2907), done.


In [0]:
def download_data(cam):
  import shutil
 # shutil.rmtree('object_detection/input_imgs/blocked')
 # shutil.rmtree('object_detection/input_imgs/notblocked')

  if cam == "single":
  
# download and read in data
    zip_address = 'http://parkingdirty.com/BlockedBikeLaneTrainingSingleCam.zip'
  else:
    zip_address = 'http://parkingdirty.com/BlockedBikeLaneTrainingFull.zip'
  
    import requests, zipfile, io
    r = requests.get(zip_address)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('object_detection/input_imgs') # extract images from zip to input_imgs folder
    
    print('data downloaded successfully')

def read_resize_label_image(img, x_dim, y_dim):
  import cv2
   
  img_orig = cv2.imread(img)
 # print(img_orig.size)
 # img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2HSV)
  img_arrays = cv2.resize(img_orig, dsize=(x_dim, y_dim), interpolation=cv2.INTER_CUBIC)
  
 # if the image file name contains "not" then assigned 0, otherwise 1, so 1 is blocked, 0 is notblocked
  if img.find("not") is not -1:
      img_labels = 0
  else:
      img_labels = 1
      
  del img_orig
   
    
  return img_arrays, img_labels


def prep_data(input_shape_x = 300, input_shape_y = 300):
  import numpy as np
  # append the label at front, and assign to object
  imgs_blocked = list(map('object_detection/input_imgs/blocked/{0}'.format, os.listdir('object_detection/input_imgs/blocked')))
  imgs_notblocked = list(map('object_detection/input_imgs/notblocked/{0}'.format, os.listdir('object_detection/input_imgs/notblocked')))

  # shuffle data
  np.random.shuffle(imgs_blocked)
  np.random.shuffle(imgs_notblocked)

  print("number of blocked images: ", len(imgs_blocked))
  print("number of not blocked images: ", len(imgs_notblocked))
  print("ratio of classes: ", len(imgs_blocked)/len(imgs_notblocked))

  # separate into training (contains training and validation), and test set
  training_set = imgs_blocked[:int(round(0.8*len(imgs_blocked)))] + imgs_notblocked[:int(round(0.8*len(imgs_notblocked)))]
  print("length of training set: ", len(training_set))

  # keep 20% of the images for test set, there will also be a validation set later
  test_set = imgs_blocked[int(round(0.8*len(imgs_blocked))):] + imgs_notblocked[int(round(0.8*len(imgs_notblocked))):]
  print("length of test set: ", len(test_set))

  del imgs_blocked
  del imgs_notblocked
  
  # read and resize data
  print("begin reading and resizing data")
  training_arrays, training_labels = zip(*[(read_resize_label_image(i, input_shape_x, input_shape_y)) for i in training_set])
  test_arrays, test_labels = zip(*[(read_resize_label_image(i, input_shape_x, input_shape_y)) for i in test_set])
  
  print("finish data reading and resizing")
  del training_set
  del test_set
  
  import pandas
  print("training: ", pandas.Series(training_labels).value_counts())
  print("test: ", pandas.Series(test_labels).value_counts())

  import numpy as np
  
  # convert images and labels into numpy arrays 

  X_train_array = np.array(training_arrays)
  Y_train_labels = np.array(training_labels)

  del training_arrays
  del training_labels

  X_test_array = np.array(test_arrays)
  Y_test_labels = np.array(test_labels)

#  del test_arrays
#  del test_labels
  
  # separate training data into training and validation sets
  # we also have the test array that was held out earlier

  from sklearn.model_selection import train_test_split

  print("splitting training data into training and validation")
  X_train, X_val, Y_train, Y_val = train_test_split(X_train_array, Y_train_labels, test_size = 0.2, 
                                                     shuffle = True)
  
  X_train //= 255
  X_val //= 255
 

  # confirm the size of the datasets
  print("training images: ", len(X_train))
  print("training labels: ", len(Y_train))
  print("validation images: ", len(X_val))
  print("validation labels: ", len(Y_val))
  
  print("data preparation complete")
  
  return X_train, X_val, Y_train, Y_val, test_arrays, test_labels




def create_model(img_dim_x, img_dim_y):
  from tensorflow.keras import applications
  conv_base = applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(200,200,3))
  conv_base.trainable = False
  # define model to transfer learning from
  # model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_dim_x, img_dim_y, 3))
  model = models.Sequential()
  model.add(conv_base)

  model.add(layers.Flatten())
  model.add(layers.Dense(1024, activation="relu"))
 # model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1024, activation="relu"))
  model.add(layers.Dense(1, activation="sigmoid"))
  
#  print(predictions)
  
#  model_final = Model(input = model.input, output = predictions)

  # basic model 
  #model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(img_dim_x, img_dim_y, 3)))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  #model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Flatten())
  #model.add(layers.Dense(128, activation='relu'))
  #model.add(layers.Dense(1, activation='sigmoid'))  #Sigmoid function at the end because we have just two classes

  return model


# create datagenerators for training and validation data

def create_datagenerator():
  train_datagen = ImageDataGenerator(rotation_range=40, # do not need rescale because we did it earlier
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,)
  
  val_datagen = ImageDataGenerator(rescale=1./255)  #We do not augment validation data. we only perform rescale

  return train_datagen, val_datagen



def train_model(img_dim_x, img_dim_y, train_datagen, val_datagen, X_train, Y_train, X_val, Y_val, batch_size = 64, epochs = 10):
  
  model = create_model(img_dim_x, img_dim_y)
  
  ## Compiler Includes Optimizer, and Learning Rate (LR), and Metrics

  model.compile(loss='binary_crossentropy', 
                optimizer=optimizers.RMSprop(lr=1e-4), 
                metrics=['acc'])

  logdir = os.path.join("logs/fit/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)
  
  early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

  model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=batch_size),
                      validation_data = val_datagen.flow(X_val, Y_val, batch_size=batch_size),
                      steps_per_epoch = X_train.shape[0] // batch_size,
                      validation_steps = X_val.shape[0] // batch_size,
                      epochs = 5,
                      callbacks=[tensorboard_callback, early])


In [3]:
# read and resize images, also store as X, Y with X as image array and Y as label
#load_modules()
download_data(cam = all)

img_arrays = [] # images
img_labels = [] # labels

img_dim_x, img_dim_y  = 200, 200

X_train, X_val, Y_train, Y_val, test_arrays, test_labels = prep_data(img_dim_x, img_dim_y)

data downloaded successfully
number of blocked images:  4131
number of not blocked images:  3542
ratio of classes:  1.166290231507623
length of training set:  6139
length of test set:  1534
begin reading and resizing data
finish data reading and resizing
training:  1    3305
0    2834
dtype: int64
test:  1    826
0    708
dtype: int64
splitting training data into training and validation
training images:  4911
training labels:  4911
validation images:  1228
validation labels:  1228
data preparation complete


In [0]:
#%tensorboard --logdir logs/fit



In [0]:
train_datagen, val_datagen = create_datagenerator()

train_model(img_dim_x, img_dim_y, train_datagen, val_datagen, X_train, Y_train, X_val, Y_val, batch_size = 64, epochs = 5)

In [0]:
## adjust model to increase capacity, currently sitting at 60% val accuracy

def model_hyp(train_datagen, X_train, Y_train, X_val, Y_val):
    from tf.keras.applications import InceptionResNetV2
    
    conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))
    conv_base.trainable = False
    
    batch_size = 32

    model = Sequential()
    model.add(conv_base)
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer= {{choice(['rmsprop', 'adam'])}},
                  metrics=['accuracy'])
    
    model.fit_generator(train_datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        epochs = 50,
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        validation_data=(X_val, Y_val))
        
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    
    print('Val accuracy:', acc)
    print('optimizer:', model.optimizer)

    
    K.clear_session()

    return {'loss': -acc, 'status': STATUS_OK}
  
if __name__ == '__main__':

  best_run, best_model = optim.minimize(model=model_hyp,
                                        data=data,
                                        algo=tpe.suggest,
                                        max_evals=30,
                                        trials=Trials(),
                                        notebook_name='hyperas')